# OpenMASTER - correlations

-------------------------

In [1]:
import pyomo.environ as pyo
import openMASTER
import numpy as np
import itertools
import pandas as pd

### Define the abstract model

In [2]:
m = openMASTER.make_model()
m

### Model data upload

* If you haven't created the .csv files, please:
    * Be aware the openMASTER_Data.xlsx file has to be downloaded using git-lfs or the following link:
        https://github.com/IIT-EnergySystemModels/openMASTER/raw/main/data/input/openMASTER_Data.xlsx?download=
    * Run the first line of code in this cell, which will both create the .csv files and load them into the DataPortal (this whole function takes several minutes)

* On the contrary, if you have already created the .csv files from the Excel file and haven't changed them, you can directly go on to the second line of code. This will save some minutes.

In any case, add "#" in front of the line you are not running.

In [3]:
data = openMASTER.load_dataportal_from_excel('../data/input/openMASTER_Data.xlsx')
#data = openMASTER.load_dataportal_from_csv()

### Principal components analysis (PCA)

In [5]:
csv_mean   = '..\data\input\mean.csv'
df_mean    = pd.read_csv(csv_mean, delimiter=';', header=[0])
X_mean     = df_mean.values


csv_data   = '..\data\input\costs_correl.csv'
df_data    = pd.read_csv(csv_data, delimiter=';', header=[0])
X_data     = df_data.values
X_o        = X_data - np.mean(X_data, axis=0)

X_z        = X_o / np.std (X_o, axis=0)
#X_z        = np.nan_to_num(X_z, nan=0.0)

# Encontrar las columnas con valores NaN en X_z
columns_with_nan = np.isnan(X_z).any(axis=0)

# Eliminar las columnas con valores NaN en X_z
X_z_clean = X_z[:, ~columns_with_nan]

# Eliminar las mismas columnas en X_o
X_o_clean = X_o[:, ~columns_with_nan]

print(X_o_clean)

#csv_covar  = '..\data\input\covar.xlsx'
#sheet_name = 'covar'
#df_covar   = pd.read_excel(csv_covar, sheet_name=sheet_name, index_col=0)
#X_covar    = df_covar.values


f_set   = data['f']
unc_set = data['sUnc']


# Compute the mean for each column

#s_bar_dict = {unc: s_bar[i-1] for i, unc in enumerate(unc_set)}

#usamos cov de la serie de datos histórica, pero centrados en la media de los valores nominales para el año 2030
#new_data = np.random.multivariate_normal(np.ravel(X_mean), X_covar, size=10000)
#new_data_o = new_data - np.ravel(X_mean)

s_bar_dict = {unc: np.ravel(X_mean)[i] for i, unc in enumerate(unc_set)}


from sklearn.decomposition import PCA
#we compute the singular value decomposition

u1, s1, vh1 = np.linalg.svd(np.cov(X_o.T), full_matrices=True)
pca1 = PCA()
pca1.fit(X_o)

u, s, vh = np.linalg.svd(np.cov(X_z.T), full_matrices=True)
pca2 = PCA()
pca2.fit(X_z)

# Comparar los componentes principales obtenidos de ambos métodos
print("Componentes principales obtenidos por el método 1:")
print(pca1.components_)
print("\nComponentes principales obtenidos por el método 2:")
print(pca2.components_)

# Comprobación de similitud
similarity = np.allclose(pca1.components_, pca2.components_)
print("\n¿Los componentes principales son similares?", similarity)


##np.linalg.svd --> Singular Value Decomposition:
##np.cov --> estimate the covariance matrix
##u is eigenvector matrix and s are the eigenvalues

#now we compute the vectors needed to build the uncertainty region
w_max = np.max(np.dot(X_o, u) / np.linalg.norm(u, axis=0), 0)
w_max_dict = {f_i: w_max[i-1] for i, f_i in enumerate(f_set)}

w_min = np.min(np.dot(X_o, u) / np.linalg.norm(u, axis=0), 0)
w_min_dict = {f_i: w_min[i-1] for i, f_i in enumerate(f_set)}

alpha_up = (w_max / np.linalg.norm(u, axis=0)) * u
alpha_up_dict = {(f_i, unc): alpha_up[i, j] for i, f_i in enumerate(f_set) for j, unc in enumerate(unc_set)}

alpha_do = (w_min / np.linalg.norm(u, axis=0)) * u
alpha_do_dict = {(f_i, unc): alpha_do[i, j] for i, f_i in enumerate(f_set) for j, unc in enumerate(unc_set)}

rest = ((w_max + w_min) / (2 * np.linalg.norm(u, axis=0))) * u
rest_dict = {(f_i, unc): rest[i, j] for i, f_i in enumerate(f_set) for j, unc in enumerate(unc_set)}


data._data[None][m.pS.name]        = s_bar_dict
data._data[None][m.pW_max.name]    = w_max_dict
data._data[None][m.pW_min.name]    = w_min_dict
data._data[None][m.pAlpha_up.name] = alpha_up_dict
data._data[None][m.pAlpha_do.name] = alpha_do_dict
data._data[None][m.pRest.name]     = rest_dict

#print(s_bar_dict)

[[-1.04795226e+00 -5.65824031e+00 -4.68260004e+00 -1.04196591e+01
   9.85916667e+00 -5.13000000e+00 -5.13000000e+00 -5.13000000e+00
  -5.13000000e+00 -2.72900000e+03  1.14300000e+03  2.87601012e+03
   1.64231053e+03  2.79132632e+03  3.38000000e+02  1.11333902e+03
   1.49537815e+02  4.92562974e+02 -2.24421953e+02 -1.06514200e+03
  -2.65224328e+03 -2.24421953e+02  3.66000000e+02  1.12100000e+03
   1.22687300e+03  6.86000000e+02  6.86000000e+02 -5.97092000e+02
   1.38700000e+03  2.43515635e+03  3.13000000e+02  4.74716666e+02
   4.84809782e+02  7.12075000e+02]
 [ 4.92572327e-01 -7.01216418e+00 -1.68682462e+01 -1.91241814e+01
  -1.67075000e+01 -7.13000000e+00 -7.13000000e+00 -7.13000000e+00
  -7.13000000e+00 -3.92900000e+03 -6.70000000e+01 -1.68585020e+02
  -9.62684215e+01 -1.63621053e+02 -7.20000000e+01 -2.37160976e+02
  -1.12310924e+02 -3.69941228e+02 -4.41171953e+02 -9.16322000e+02
  -2.30075393e+03 -4.41171953e+02  2.60000000e+01  6.71000000e+02
   5.91503000e+02  4.56000000e+02  4.5600

C:\Users\afrmatas\AppData\Local\Temp\ipykernel_11816\404563478.py:11: RuntimeWarning: invalid value encountered in divide
  X_z        = X_o / np.std (X_o, axis=0)


LinAlgError: SVD did not converge

### Create the instance of the abstract model 

In [ ]:
instance = m.create_instance(data)

In [ ]:
#data._data[None][m.pCECapex.name]
#instance.pS.pprint()

### Solve the model instance

To solve the model instance, please select a solver within the Pyomo SolverFactory. Please note that any solver has to be previously installed.

In [ ]:
solver = pyo.SolverFactory('gurobi')

solver_results = solver.solve(instance, keepfiles=False, tee=True)

### **Results**

Extract information on variables through the model output to .csv files

In [ ]:
path        = "../data/input/openMASTER_Data.xlsx"
output_path = "../data/tmp/output"
sheetname   = "Output"

d_vars_from_instance = openMASTER.export_model_to_csv(path, output_path, sheetname, instance)

Loading variable information from .csv to a dictionary containing all outputs

In [ ]:
d_vars = openMASTER.import_results_from_csv(output_path)